In [1]:
import pandas as pd


In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = dataset.drop("User ID", axis=1)


In [4]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent = dataset[['Age', 'EstimatedSalary',  'Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dependent = dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.30)

In [10]:
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
x_train = scx.fit_transform(x_train)
x_test = scx.transform(x_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    "criterion":["gine", "entrophy", "log_loss"],
    "max_features": ["sqrt", "log2", None],
    "n_estimators": [10, 100]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring="f1_weighted")

grid.fit(x_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_par

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gine', 'entrophy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred = grid.predict(x_test)

In [13]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, y_pred, average="weighted")
print("f1 macro value for the best parameter is {}".format(grid.best_params_), f1_macro)

f1 macro value for the best parameter is {'criterion': 'log_loss', 'max_features': 'log2', 'n_estimators': 100} 0.925212953725731


In [14]:
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])
print("roc_auc score for the model is {}:".format(roc_score))

roc_auc score for the model is 0.9614078419265205:


In [15]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test, y_pred)
clf_report=classification_report(y_test, y_pred)
print(clf_report)

print(cm)


              precision    recall  f1-score   support

           0       0.95      0.94      0.94        79
           1       0.88      0.90      0.89        41

    accuracy                           0.93       120
   macro avg       0.91      0.92      0.92       120
weighted avg       0.93      0.93      0.93       120

[[74  5]
 [ 4 37]]


In [16]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004800,0.003919,0.000000,0.000000,gine,sqrt,10,"{'criterion': 'gine', 'max_features': 'sqrt', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
1,0.000502,0.001004,0.000000,0.000000,gine,sqrt,100,"{'criterion': 'gine', 'max_features': 'sqrt', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
2,0.002428,0.003048,0.000000,0.000000,gine,log2,10,"{'criterion': 'gine', 'max_features': 'log2', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
3,0.002238,0.003135,0.000000,0.000000,gine,log2,100,"{'criterion': 'gine', 'max_features': 'log2', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
4,0.001600,0.003200,0.000000,0.000000,gine,None,10,"{'criterion': 'gine', 'max_features': None, 'n...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
5,0.001602,0.002210,0.000000,0.000000,gine,None,100,"{'criterion': 'gine', 'max_features': None, 'n...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
6,0.001600,0.003200,0.000000,0.000000,entrophy,sqrt,10,"{'criterion': 'entrophy', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,0.000000,0.000000,0.000000,0.000000,entrophy,sqrt,100,"{'criterion': 'entrophy', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
8,0.001600,0.003199,0.000000,0.000000,entrophy,log2,10,"{'criterion': 'entrophy', 'max_features': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
9,0.001565,0.003130,0.000000,0.000000,entrophy,log2,100,"{'criterion': 'entrophy', 'max_features': 'log...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
